In [2]:
import numpy as np
x = np.array([
    [1, 1, 1],
    [2, 2, 2],
    [3, 3, 3]
])

a, b, c = x

print(c)

[3 3 3]


In [6]:
from scipy import stats

std = 0.9
z_diff = np.array([1, 2])
P = stats.norm.cdf(z_diff/std)
print(P)


[0.86673974 0.98686585]
